<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GLM4DOT6_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://z.ai/manage-apikey/subscription

In [2]:
from google.colab import userdata
ZAI_KEY_API = userdata.get('ZAI_KEY')


In [4]:
from openai import OpenAI

client = OpenAI(
    api_key=ZAI_KEY_API,
    base_url="https://api.z.ai/api/paas/v4/"
)

completion = client.chat.completions.create(
    model="glm-4.6",
    messages=[
        {"role": "system", "content": "You are a smart and creative novelist"},
        {"role": "user", "content": "Please write a short fairy tale story as a fairy tale master"}
    ]
)

print(completion.choices[0].message.content)


Once, in a kingdom tucked between the memory of an old forest and the sigh of a sleeping sea, a peculiar trouble began. It was not a dragon or a war, but a quiet fraying at the edges of the world.

It started small. A baker would find that the corner of his famous spice cake had simply un-baked itself, returning to flour and dust. A gardener would watch a rosebud unfurl not into a bloom, but into a spool of silken thread. Soon, whole things were coming undone: the hem of a meadow would fray into nothingness, a cobblestone path would forget its order and become a scattering of pebbles, and the evening star would sometimes flicker out like a snuffed candle.

The King, a man who loved his kingdom to be as neat and orderly as a chessboard, was distraught. He sent his boldest knights to the borders to "sew the edges shut," but their swords simply passed through the emptiness. He summoned his most powerful wizards to "cast a spell of mending," but their incantations dissolved into meaningle

## AGENTIC

In [5]:
import json
import os
# Import Colab's user data utility for secure API key handling
from google.colab import userdata
from openai import OpenAI

# --- 0. Configuration (Securely Load API Key) ---
# NOTE: You must first save your Z.ai API key in Colab Secrets as 'ZAI_KEY'
try:
    ZAI_KEY_API = userdata.get('ZAI_KEY')
    if not ZAI_KEY_API:
        raise ValueError("ZAI_KEY not found in Colab secrets.")
except Exception as e:
    print(f"Error loading API key: {e}. Please ensure 'ZAI_KEY' is set in Colab Secrets.")
    # Exit or use a placeholder to prevent execution with a bad key
    ZAI_KEY_API = "PLACEHOLDER_ERROR_KEY"

# Initialize the OpenAI-compatible client for Z.ai
client = OpenAI(
    api_key=ZAI_KEY_API,
    base_url="https://api.z.ai/api/paas/v4/"
)

# --- 1. Tool (Function) Definitions ---

# Define the actual Python functions the agent can call
def get_current_stock_price(ticker_symbol: str) -> str:
    """
    Retrieves the latest stock price for a given ticker symbol.
    In a real application, this would query a live finance API.
    """
    ticker = ticker_symbol.upper()
    if ticker == "TSLA":
        # Simulate a real-time data response
        return json.dumps({"ticker": ticker, "price": "298.50", "currency": "USD"})
    elif ticker == "AAPL":
        return json.dumps({"ticker": ticker, "price": "189.15", "currency": "USD"})
    else:
        # The agent should be able to handle tool failure/missing data
        return json.dumps({"error": f"Stock price for {ticker} not found."})

# Map function names to the actual functions
available_functions = {
    "get_current_stock_price": get_current_stock_price,
}

# Define the function schema (specifications) for the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_stock_price",
            "description": "Get the current stock price for a given company ticker.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker_symbol": {
                        "type": "string",
                        "description": "The stock ticker symbol (e.g., TSLA or AAPL).",
                    },
                },
                "required": ["ticker_symbol"],
            },
        },
    }
]

# --- 2. Agentic Execution Logic (The Loop) ---

def run_agentic_query(client, user_prompt):
    """Executes the multi-step agentic workflow."""
    if ZAI_KEY_API == "PLACEHOLDER_ERROR_KEY":
        return "Cannot run agent: API Key not loaded securely from Colab Secrets."

    messages = [
        {"role": "user", "content": user_prompt}
    ]

    # Step 1: Initial call - The model decides to respond or call a tool
    print("Agent: Thinking/Planning...")
    response = client.chat.completions.create(
        model="glm-4.6",
        messages=messages,
        tools=tools,
        tool_choice="auto", # Allows the model to choose the function
    )

    response_message = response.choices[0].message

    # Check if the model has generated a function call (Agent's Plan)
    if response_message.tool_calls:
        print(f"Agent: Calling tool(s) as planned...")

        # Add the model's call request to the conversation history
        messages.append(response_message)

        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions.get(function_name)

            if function_to_call:
                function_args = json.loads(tool_call.function.arguments)

                # Step 2: EXECUTE the tool (function) and get the real-world result
                function_output = function_to_call(
                    **function_args
                )

                # Step 3: Send the tool's output back to the model for reflection
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_output, # The tool's result is returned here
                    }
                )

        # Final call: The model synthesizes the answer from the tool's output (Reflection)
        print("Agent: Synthesizing final answer from tool data...")
        final_response = client.chat.completions.create(
            model="glm-4.6",
            messages=messages,
        )
        return final_response.choices[0].message.content

    # Handle case where no tool call was needed (simple response)
    return response_message.content

# --- 3. Execute the Agent ---

user_prompt = "What are the stock prices for Tesla (TSLA) and Apple (AAPL)?"

# You can see the step-by-step thinking printed above the final output
agent_output = run_agentic_query(client, user_prompt)

print("\n--- FINAL AGENT REPORT ---")
print(f"User Prompt: {user_prompt}")
print(f"Final Answer: {agent_output}")

Agent: Thinking/Planning...
Agent: Calling tool(s) as planned...
Agent: Synthesizing final answer from tool data...

--- FINAL AGENT REPORT ---
User Prompt: What are the stock prices for Tesla (TSLA) and Apple (AAPL)?
Final Answer: 
Here are the current stock prices for Tesla and Apple:

**Tesla (TSLA)**: $298.50 USD

**Apple (AAPL)**: $189.15 USD

Please note that stock prices fluctuate throughout trading hours, so these prices reflect the current market value at the time of this query.


## CODING

In [7]:
import json
import os
from openai import OpenAI

# --- 0. Configuration (Securely Load API Key) ---
# NOTE: You must first save your Z.ai API key in Colab Secrets as 'ZAI_KEY'
try:
    ZAI_KEY_API = userdata.get('ZAI_KEY')
    if not ZAI_KEY_API:
        raise ValueError("ZAI_KEY not found in Colab secrets.")
except Exception as e:
    print(f"Error loading API key: {e}. Please ensure 'ZAI_KEY' is set in Colab Secrets.")
    # Exit or use a placeholder to prevent execution with a bad key
    ZAI_KEY_API = "PLACEHOLDER_ERROR_KEY"

# Initialize the OpenAI-compatible client for Z.ai
client = OpenAI(
    api_key=ZAI_KEY_API,
    base_url="https://api.z.ai/api/paas/v4/"
)

# --- 1. Tool (Function) Definition for the Agent ---

# This Python function defines the backend logic (the calculation tool)
def calculate_discounted_price(base_price: float, discount_rate: float) -> str:
    """
    Calculates the final price after applying a discount rate.
    Returns the result as a formatted string with two decimal places.
    """
    final_price = base_price * (1 - discount_rate)
    return f"${final_price:,.2f}"

# --- 2. Coding Agent Prompt ---

system_prompt = (
    "You are an expert Full-Stack AI Coding Agent. Your task is to generate clean, "
    "complete, and functional code for the user's request. The entire output must be "
    "one single markdown block containing the HTML, CSS, and JavaScript. "
    "You MUST include the JavaScript implementation of the 'calculate_discounted_price' function "
    "using the logic: finalPrice = basePrice * (1 - discountRate), and use it to set the final price."
)

user_prompt = """
Create a single-file HTML page for an e-commerce product card.
The card must display:
1.  **Product Name:** 'AI Agent Framework License'
2.  **Base Price:** $199.99
3.  **Discount Rate:** 15% (0.15)

The JavaScript must implement a function called `calculateDiscountedPrice(basePrice, discountRate)` to compute the final price.
Use this function to dynamically calculate and display the final price on the page using a DOM manipulation method.
Use modern CSS (embedded in a `<style>` tag) for a visually polished appearance.
"""

# --- 3. Execute the Coding Agent ---

print("Coding Agent: Requesting GLM-4.6 to generate code...")

try:
    if ZAI_KEY_API == "PLACEHOLDER_ERROR_KEY":
        raise Exception("API Key Error: Cannot proceed without a valid key.")

    completion = client.chat.completions.create(
        model="glm-4.6",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.2 # Low temperature for accurate code generation
    )

    print("\n--- GENERATED CODE OUTPUT ---")
    print(completion.choices[0].message.content)

except Exception as e:
    print(f"\nAPI Execution Error: {e}")
    print("\nCheck your network connection and ensure your 'ZAI_KEY' is valid and has access to the 'glm-4.6' model.")

Coding Agent: Requesting GLM-4.6 to generate code...

--- GENERATED CODE OUTPUT ---

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI Agent Framework License - Product Card</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, 'Helvetica Neue', Arial, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            display: flex;
            justify-content: center;
            align-items: center;
            padding: 20px;
        }

        .product-card {
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 60px rgba(0, 0, 0, 0.3);
            max-width: 400px;
            width: 100%;
            ov